In [ ]:
import pandas 
import yfinance
raw_data = yfinance.download(tickers = "VOW3.DE, PAH3.DE, BMW.DE",
                             interval = "1d", group_by='ticker', auto_adjust= True, treads = True)

[*********************100%***********************]  3 of 3 completed


In [ ]:
raw_data.isna().sum()

PAH3.DE  Open      3165
         High      3165
         Low       3165
         Close     3165
         Volume    3165
VOW3.DE  Open       443
         High       443
         Low        443
         Close      443
         Volume     443
BMW.DE   Open         0
         High         0
         Low          0
         Close        0
         Volume       0
dtype: int64

In [ ]:
raw_data=raw_data.asfreq("b")
raw_data = raw_data.fillna(method = "bfill")

In [ ]:
raw_data["BMW.DE"].describe()

,Open,High,Low,Close,Volume
count,6791.000000,6791.000000,6791.000000,6791.000000,6.791000e+03
mean,36.864318,37.300372,36.384012,36.849278,2.187155e+06
std,21.847711,22.034506,21.633386,21.840361,1.478616e+06
min,9.328610,9.348643,9.142151,9.293167,0.000000e+00
25%,18.663511,18.926166,18.346717,18.637691,1.285535e+06
50%,25.310211,25.574000,25.033846,25.304018,1.883699e+06
75%,57.296951,57.845506,56.628138,57.241077,2.700980e+06
max,92.837648,93.339571,91.759437,92.317131,2.871954e+07


In [ ]:
start_date_2010 = "2010-01-01"
end_date_2010 = "2010-12-31"
start_date_2011 = "2011-01-01"
end_date_2011 = "2011-12-31"

In [ ]:
raw_data['VOW3.DE'][start_date_2010:end_date_2010].Close.corr(raw_data['PAH3.DE'][start_date_2010:end_date_2010].Close)


0.6499594499811755

In [ ]:
raw_data['VOW3.DE'][start_date_2011:end_date_2011].Close.corr(raw_data['PAH3.DE'][start_date_2011:end_date_2011].Close)

0.5991603661880602

In [ ]:
start_date_2009 = "2009-01-01"
end_date_2009 = "2009-12-31"

In [ ]:
from pmdarima.arima import auto_arima
mod_pr_pour_2011 = auto_arima(raw_data['VOW3.DE'][start_date_2011:end_date_2011].Close,
                              exogenous = raw_data[['PAH3.DE','BMW.DE']][start_date_2009:end_date_2009],
                              m=5,
                              max_p = 5,
                              max_q = 5)

In [ ]:
mod_pr_pour_2009.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  261
Model:               SARIMAX(1, 1, 0)   Log Likelihood                -350.950
Date:                Mon, 21 Nov 2022   AIC                            705.900
Time:                        07:41:06   BIC                            713.022
Sample:                    01-01-2009   HQIC                           708.763
                         - 12-31-2009                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1495      0.057      2.608      0.009       0.037       0.262
sigma2         0.8708      0.037     23.538      0.000       0.798       0.943
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               483.15
Prob(Q):                              0.97   Prob(JB):                         0.00
Heteroskedasticity (H):               0.56   Skew:                            -0.67
Prob(H) (two-sided):                  0.01   Kurtosis:                         9.54
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
raw_data['ret_vol']= raw_data['VOW3.DE'].Close.pct_change(1).mul(100)
raw_data['ret_por']= raw_data['PAH3.DE'].Close.pct_change(1).mul(100)
raw_data['ret_bmw']= raw_data['BMW.DE'].Close.pct_change(1).mul(100)

from pmdarima.arima import auto_arima
mod_pr_pour_2009 = auto_arima(raw_data['ret_vol'][start_date_2009:end_date_2009],
                              exogenous = raw_data[['ret_por','ret_bmw']][start_date_2009:end_date_2009],
                              m=5,
                              max_p = 5,
                              max_q = 5)

In [ ]:
mod_pr_pour_2009.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  261
Model:                        SARIMAX   Log Likelihood                -706.996
Date:                Mon, 21 Nov 2022   AIC                           1415.993
Time:                        07:37:11   BIC                           1419.557
Sample:                    01-01-2009   HQIC                          1417.425
                         - 12-31-2009                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        13.1934      0.772     17.092      0.000      11.680      14.706
===================================================================================
Ljung-Box (L1) (Q):                   0.16   Jarque-Bera (JB):                67.74
Prob(Q):                              0.69   Prob(JB):                         0.00
Heteroskedasticity (H):               0.55   Skew:                             0.02
Prob(H) (two-sided):                  0.01   Kurtosis:                         5.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""